# NYC taxi visualization

In this tutorial, we were provided with a dataset which contains information about the number of taxi ride drop-offs in New York City in the year 2015.
The raw data (obtained from the [Vaex documentation](https://vaex.readthedocs.io/en/latest/datasets.html))
has been histogrammed into three dimensions: latitude, longitude and hour-of-the-day.

We will first load the data and inspect its contents:

In [ ]:
%matplotlib widget
import scipp as sc
import plopp as pp
from plopp import widgets
from plopp.data import examples
import ipywidgets as ipw

data = sc.io.open_hdf5(examples.nyc_taxi())
data

The goal of this tutorial is to create an interface with the following:

- a central 2D image that shows a longitude-latitude map of the drop-offs, for a single hour-of-the-day slice.
- a slider above the main panel which allows to navigate the hour-of-the-day dimension, updating the 2D map.
- a 1D figure below the 2D map, displaying the same data but summed along the latitude dimension.
- in the 1D panel, we also want to add a gaussian smoothed version of the 1D curve.

The first step is to create an `input_node` for the original data,
a `widget_node` that will provide the index to be used to select the hour-of-the-day,
and a `slice_node` node that slices the original data according to the index of the slider.

In [ ]:
# Input data node
in_node = pp.input_node(data)
in_node.name = 'Input data'

# Slider node that provides index to slice
slider = ipw.IntSlider(min=0, max=23)
slider_node = pp.widget_node(slider)
slider_node.name = 'Slider'

# Node that actually does the slicing
slice_node = pp.node(lambda da, ind: da['hour', ind])(da=in_node, ind=slider_node)
slice_node.name = 'Slicing node'

pp.show_graph(in_node)

Next, we add a two-dimensional figure to the bottom node that will display the latitude/longitude map:

In [ ]:
# Add figure to the bottom node
fig2d = pp.figure2d(slice_node, norm='log')
pp.show_graph(in_node)

We can now add a node that performs a sum along the latitude dimension

In [ ]:
# Add a node that performs sum along the latitude dimension
sumx = pp.node(sc.sum, dim='latitude')(slice_node)
sumx.name = 'Sum along latitude'

pp.show_graph(in_node)

We next add a smoothing node that will smooth the 1d data below the `slice_node`

In [ ]:
from scipp.ndimage import gaussian_filter

# Add a smoothing node that will smooth the 1d data
smooth = pp.node(gaussian_filter, sigma=5)(sumx)
smooth.name = 'Gaussian smoothing'

pp.show_graph(in_node)

Finally, we add the bottom panel figure, giving it both the `sumx` and `smooth` nodes as inputs,
to display both curves on the same axes:

In [ ]:
# Add figure that plots both smoothed and unsmoothed data
fig1d = pp.figure1d(sumx, smooth, norm='log')

pp.show_graph(in_node)

### Putting it all together

In [ ]:
in_node = pp.input_node(data)  # Input data node

# Slider node that provides index to slice
slider = ipw.IntSlider(min=0, max=23)
slider_node = pp.widget_node(slider)

# Node that actually does the slicing
slice_node = pp.node(lambda da, ind: da['hour', ind])(da=in_node, ind=slider_node)

fig2d = pp.figure2d(slice_node, norm='log') # Add 2D map

sumx = pp.node(sc.sum, dim='latitude')(slice_node)  # Sum along latitude

smooth = pp.node(gaussian_filter, sigma=5)(sumx)  # Gaussian smooth

fig1d = pp.figure1d(sumx, smooth, norm='log')  #  Add 1D figure

widgets.Box([slider, fig2d, fig1d])  # Container box

Now when we interact with the slider, the 2D map, as well as both curves on the 1D profile will update.